# Working example of GLAD 
Fitting GLAD on a single random sparse graph with samples obtained from a corresponding multivariate Gaussian distribution

In [10]:
# importing required modules: Python 3 used. 
import numpy as np
import pandas as pd
import torch
print('Pytorch version ', torch.__version__)
import sys
import scipy
import networkx as nx
print('networkx version ', nx.__version__)
import copy, random
import matplotlib.pyplot as plt
from glad import glad
from glad_model import glad_model
import torch.nn as nn
from metrics import report_metrics
import warnings
warnings.filterwarnings('ignore')

Pytorch version  1.4.0
networkx version  2.5


In [2]:
# initializing basic params
D = 10 # Dimension of graph
sparsity = 0.2 # erdos-renyi sparsity of true precision matrix
w_max, w_min = 0.5, 1 # sample true precision matrix entries ~U[w_min, w_max] 
sample_size = 500 # Number of samples from the true graph
lr_glad = 0.03 # Learning rate of GLAD
use_optimizer = 'adam' # Optimizer
INIT_DIAG = 0 # Initialize as (S + theta_init_offset * I)^-1
lambda_init = 1 # Initial lambda value
L = 15 # Number of unrolled iterations

## Preparing the input data & output precision matrix

In [3]:
# initializing precision matrix
def get_sparsity_pattern(seed=None):
    if seed != None:
        np.random.seed(seed)
    prob = sparsity
    G = nx.generators.random_graphs.gnp_random_graph(D, prob, seed=seed, directed=False)
    edge_connections = nx.adjacency_matrix(G).todense() # adjacency matrix
    return edge_connections

# Get the input data and corresponding output true precision matrix for training
def get_samples(edge_connections, seed=None, u=0.1): # mean = 0
    mean_value = 0 # zero mean of Gaussian distribution
    mean_normal = np.ones(D) * mean_value
    if seed != None:
        np.random.seed(seed)
        
    # uniform [w_min, w_max]
    U = np.matrix(np.random.random((D, D)) * (w_max - w_min) + w_min)
    theta = np.multiply(edge_connections, U)
    # making it symmetric
    theta = (theta + theta.T)/2 + np.eye(D)
    smallest_eigval = np.min(np.linalg.eigvals(theta))
    # Just in case : to avoid numerical error in case a epsilon complex component present
    smallest_eigval = smallest_eigval.real
    # making the min eigenvalue as u
    precision_mat = theta + np.eye(D)*(u - smallest_eigval)
    print('Smallest eigenvalue = ', np.min(np.linalg.eigvals(precision_mat)))
    
    cov = np.linalg.inv(precision_mat) # avoiding the use of pinv as data not true representative of conditional independencies.
    # get the samples 
    if seed != None:
        np.random.seed(seed)
    # Sampling data from multivariate normal distribution
    data = np.random.multivariate_normal(mean=mean_normal, cov=cov, size=sample_size)
    print('data: ', data.shape, ' theta: ', theta.shape)
    return data, precision_mat # MxD, DxD

In [4]:
edge_connections = get_sparsity_pattern() # the adjacency matrix
X, theta_true = get_samples(edge_connections) # input data = X, output true precision matrix = theta_true
#print('Check: ', X, theta_true)

Smallest eigenvalue =  0.09999999999999937
data:  (500, 10)  theta:  (10, 10)


## Training the glad

In [5]:
# helper functions 
USE_CUDA = False
def convert_to_torch(data, TESTING_FLAG=False, USE_CUDA=USE_CUDA):# convert from numpy to torch variable 
    if USE_CUDA == False:
        data = torch.from_numpy(data.astype(np.float, copy=False)).type(torch.FloatTensor)
        if TESTING_FLAG == True:
            data.requires_grad = False
    else: # On GPU
        if TESTING_FLAG == False:
            data = torch.from_numpy(data.astype(np.float, copy=False)).type(torch.FloatTensor).cuda()
        else: # testing phase, no need to store the data on the GPU
            data = torch.from_numpy(data.astype(np.float, copy=False)).type(torch.FloatTensor).cuda()
            data.requires_grad = False
    return data

def get_optimizers(model_glad):
    lrG = lr_glad
    if use_optimizer == 'adam':
        optimizer_glad = torch.optim.Adam(model_glad.parameters(), lr=lrG, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)
    else:
        print('Optimizer not found!')
    return optimizer_glad
criterion_graph = nn.MSELoss()
# Consider that all the data can be shifted to GPU
X = convert_to_torch(X)
theta_true = convert_to_torch(theta_true, TESTING_FLAG=True)

## Training GLAD using supervision

In [6]:
# Initialize the model
model_glad = glad_model(L=L, theta_init_offset=0.1, nF=3, H=3, USE_CUDA=USE_CUDA)
optimizer_glad = get_optimizers(model_glad)
# get the sample covariance matrix for GLAD: Note, adjust the mean if needed
S = torch.matmul(X.t(), X)/X.shape[0] # DxD matrix
for epoch in range(40):
    optimizer_glad.zero_grad()
    theta_pred, loss = glad(S, theta_true, model_glad, [D, INIT_DIAG, lambda_init, L],criterion_graph)
#    print(theta_true.shape, theta_pred.shape)
    loss.backward() # calculate the gradients
    optimizer_glad.step() # update the weights
    compare_theta = report_metrics(np.array(theta_true), theta_pred[0].detach().numpy())
    print('epoch: ', epoch, ' loss: ', loss.detach().numpy()[0])
    if epoch %5 == 0:
        print(': Recovery :FDR, TPR, FPR, SHD, nnz_true, nnz_pred, precision, recall, Fb, aupr, auc ')
        print( *np.around(compare_theta, 3))

epoch:  0  loss:  0.19684519
: Recovery :FDR, TPR, FPR, SHD, nnz_true, nnz_pred, precision, recall, Fb, aupr, auc 
nan 0.0 0.0 7.0 7.0 0.0 nan 0.0 0.0 0.156 0.5
epoch:  1  loss:  0.17134902
epoch:  2  loss:  0.14000645
epoch:  3  loss:  0.10775185
epoch:  4  loss:  0.07943447
epoch:  5  loss:  0.056306645
: Recovery :FDR, TPR, FPR, SHD, nnz_true, nnz_pred, precision, recall, Fb, aupr, auc 
0.0 1.0 0.0 0.0 7.0 7.0 1.0 1.0 1.0 1.0 1.0
epoch:  6  loss:  0.039452948
epoch:  7  loss:  0.03031966
epoch:  8  loss:  0.026799602
epoch:  9  loss:  0.021831678
epoch:  10  loss:  0.016253486
: Recovery :FDR, TPR, FPR, SHD, nnz_true, nnz_pred, precision, recall, Fb, aupr, auc 
0.0 1.0 0.0 0.0 7.0 7.0 1.0 1.0 1.0 1.0 1.0
epoch:  11  loss:  0.011662516
epoch:  12  loss:  0.008434585
epoch:  13  loss:  0.0063241594
epoch:  14  loss:  0.004983781
epoch:  15  loss:  0.004139496
: Recovery :FDR, TPR, FPR, SHD, nnz_true, nnz_pred, precision, recall, Fb, aupr, auc 
0.3 1.0 0.079 3.0 7.0 10.0 0.7 1.0 0.824 